<a href="https://colab.research.google.com/github/DattaAnnewsha/100-Days-of-RTL/blob/main/EE599_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 - EE599 Systems for Machine Learning, Fall 2023
University of Southern California

Instructors: Arash Saifhashemi, Murali Annavaram

In this homework assignment, we will ask you to use various methods to implement convolution operation, and then measure and analyze the performance of each method.

## Prepare your Google Drive
- Download `ML_Systems_HW3` zip file from GitHub and unzip the it (you may need to rename the unzipped folder).
- Upload unzipped folder to ``My Drive`` in Google Drive.

In [19]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ml-systems-hw3-DattaAnnewsha-master/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Verify that you are in the correct working directory.

In [20]:
!pwd

/content/drive/MyDrive/ml-systems-hw3-DattaAnnewsha-master/src


## Create a folder named `build` under `ML_Systems_HW3/src`, which will be used to store executable files.

In [21]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [22]:
!bash compile.csh

Compiled q1_conv2d_naive1.cc successfully!
Compiled q2_conv2d_toeplitz1.cc successfully!
Compiled q3_conv2d_toeplitz_transB1.cc successfully!
Compiled q4_conv2d_toeplitz_avx1.cc successfully!
Compiled q5_conv2d_toeplitz_avx_openmp1.cc successfully!
Compiled q6_conv2d_toeplitz_blas1.cc successfully!
All files compiled successfully!


In [23]:
!bash run.csh


Running q1_conv2d_naive1 ...
Manual Memory Usage (KB): 16706
Virtual Memory (KB): 16716
temp: 0
Time taken by convolution: 22887 milliseconds
Sum of all outputs: 47243571200
Input freed
Weight freed
Output freed
-------------------------------------------
Running q2_conv2d_toeplitz1 ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 13475 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q3_conv2d_toeplitz_transB1 ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 9859 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q4_conv2d_toeplitz_avx1 ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 2308 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q5_conv2d_toeplitz_avx_openmp1 ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function with 2 threads: 1588 milliseconds
Sum of all outputs: 47243571200
Time taken

## Compile and run your code:
* To compile your C++ code, run command `!bash compile.csh`
* To run your executable code, run command `!bash run.csh`
* Check those csh files and modify accordingly when testing your code.
* You can write and test your code on your local machine but make sure you can compile and run them on Colab. Also you should report the performance measured from Colab enverionment.

## Write code and answer all questions in this notebook.
Note that to measure the performance of each code, we want you to flush your cache. All the template files provide code that flush caches, but we need you to find the cache size of your system. Use the command below to display information about your CPU.

In [24]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               AuthenticAMD
  Model name:            AMD EPYC 7B12
    CPU family:          23
    Model:               49
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4499.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
                         od nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pn
                         i pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt a
                         es xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy 
              

## Q1
Implement 2D Convolution in the ``q1_conv2d_naive.cc`` using nested for loops. Assume batch size = 1, no padding and stride = 1. Check `util.h` file and understand what each function does. Use the micro `INDEX_4D_TO_1D` to help convert 4d index to 1d index. Measure and report runtime  (in milliseconds) and memory usage (in KB). Manually calculate memory usage and report. Does your calculation match with the measurement?

**Buffer Size:**

Input Size = 128 * 128 * 128

Weight Size = 128 * 128 * 3 * 3

Output Size = 128 * 126 * 126

**Memory usage = 16706 KB**


Note: for floationg point operation 4bytes * buffer size

In [ ]:
#include <chrono>
#include <iostream>

#include "../utils.h"

using namespace std;
using namespace std::chrono;

int main(void) {
  // Modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = 55 * 1024 * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float* cacheFlush = allocateAlignedFloatArray(cacheSize);
  bool DBG = false;
  unsigned int weight_o;
  unsigned int weight_i;
  unsigned int weight_r;
  unsigned int weight_s;

  // Dimension of the convolution input
  unsigned int input_c;
  unsigned int input_h;
  unsigned int input_w;

  // Dimension of the convolution weight
  if (DBG == false) {
    weight_o = 128;
    weight_i = 128;
    weight_r = 3;
    weight_s = 3;

    // Dimension of the convolution input
    input_c = 128;
    input_h = 128;
    input_w = 128;
  } else {
    weight_o = 3;
    weight_i = 3;
    weight_r = 2;
    weight_s = 2;

    // Dimension of the convolution input
    input_c = 1;
    input_h = 3;
    input_w = 3;
  }

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // Allocate input buffers
  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float* weight = allocateAlignedFloatArray(weightSize);
  float* input = allocateAlignedFloatArray(inputSize);

  // Initialize input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // TODO: Calculate output dimension and allocate output buffer
  const unsigned int output_p = input_h - (weight_r - 1) - 1 + 1;
  const unsigned int output_q = input_w - (weight_s - 1) - 1 + 1;
  const unsigned int output_c = weight_o;
  const unsigned int opSize = output_c * output_p * output_q;
  float* output = allocateAlignedFloatArray(opSize);
  initializeBuffer(output, opSize);

  // Calculate memory usage manually
  // Assuming each float variable consumes 4 bytes
  unsigned int memoryUsageKB = (weightSize + inputSize + opSize) * 4 / 1024;
  cout << "Manual Memory Usage (KB): " << memoryUsageKB << endl;

  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  cout << "Virtual Memory (KB): "
       << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage << endl;

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: Use nested for loops to compute convolution
  for (int m = 0; m < output_c; m++) {
    for (int q = 0; q < output_q; q++) {
      for (int p = 0; p < output_p; p++) {
        output[INDEX_4D_TO_1D(0, m, p, q, 0, output_c, output_p, output_q)] = 0.0;
        for (int c = 0; c < input_c; c++) {
          for (int s = 0; s < weight_s; s++) {
            for (int r = 0; r < weight_r; r++) {
              output[INDEX_4D_TO_1D(0, m, p, q, 0, output_c, output_p, output_q)] +=
                  input[INDEX_4D_TO_1D(0, c, p + r, q + s, 0, input_c, input_h, input_w)] *
                  weight[INDEX_4D_TO_1D(m, c, r, s, output_c, input_c, weight_r, weight_s)];
            }
          }
        }
      }
    }
  }

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by convolution: " << duration.count() << " milliseconds" << endl;

  // Sum all outputs
  float sum = 0.0;
  for (int i = 0; i < opSize; i++) {
    sum += output[i];
  }
  cout.precision(20);
  cout << "Sum of all outputs: " << sum << endl;

  // Free all allocated buffers
  deallocateAlignedFloatArray(input);
  cout << "Input freed" << endl;
  deallocateAlignedFloatArray(weight);
  cout << "Weight freed" << endl;
  deallocateAlignedFloatArray(output);
  cout << "Output freed" << endl;

  return 0;
}


## Q2
Use Img2col algorithm to convert the input matrix and kernel matrix to toeplitz form in file ``q2_conv2d_toeplitz.cc``. Interpret input toeplitz matrix and kernel toeplitz matrix as 2d matrix, and store both of them in row major. Use nested for loops to perform matrix multiplication in `matMul` function and call it in `main` function . There is another micro `INDEX_2D_TO_1D` that helps convert 2d index to 1d index. Measure and report runtime and memory usage. Manually calculate memory usage and report. Does your calculation match with the measurement?

**For rest of the toeplitz questions Buffer size:**

Toeplitz Input = 128 * 3 * 3 * 126 * 126

Toeplitz Weight = 128 * 128 * 3 * 3

Toeplitz Output = 128 * 126 * 126

***Memory usage = 79956 KB ***

In [ ]:
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

void matMul(const float* A, const float* B, float* C, int A_rows, int A_cols,
            int B_rows, int B_cols) {
  if (A_cols != B_rows) {
    // The matrices can't be multiplied if A's number of columns
    // isn't equal to B's number of rows.
    throw std::invalid_argument(
        "Matrix dimensions mismatch for multiplication");
  }

  // TODO: implement matmul using 3 loops
  for (int out_row = 0; out_row < A_rows; out_row++){
    for (int out_col = 0; out_col < B_cols; out_col++){
      C[INDEX_2D_TO_1D(out_row, out_col, A_rows, B_cols)] = 0.0;
      for (int innercnt = 0; innercnt < A_cols; innercnt++) {
        C[INDEX_2D_TO_1D(out_row, out_col, A_rows, B_cols)] += A[INDEX_2D_TO_1D(out_row, innercnt, A_rows, A_cols)] * B[INDEX_2D_TO_1D(innercnt, out_col, B_rows, B_cols)];
      }
    }
  }
}

int main(void) {
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = 55 * 1024 * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float* cacheFlush = allocateAlignedFloatArray(cacheSize);

  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float* weight = allocateAlignedFloatArray(weightSize);
  float* input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_c = weight_o;
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int output_row = weight_o;
  const unsigned int output_col = output_p * output_q;
  const unsigned int opSize = output_row * output_col;
  float* output = allocateAlignedFloatArray(opSize);

  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int new_input_h = weight_r * weight_s * weight_i;
  const unsigned int new_input_w = output_p * output_q;
  const unsigned int new_ipSize = new_input_h * new_input_w;
  float* toep_input = allocateAlignedFloatArray(new_ipSize);

  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int new_weight_r = weight_o;
  const unsigned int new_weight_s = new_input_h;
  const unsigned int new_weightSize = new_weight_r * new_weight_s;
  float* toep_weight = allocateAlignedFloatArray(new_weightSize);

  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;

  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in row
  // major order
  // toeplitz input matrix in row-major
  int row = 0;
  for (int inc = 0; inc < weight_i; inc++){
    for (int r = 0; r < weight_r; r++) {
      for (int s = 0; s < weight_s; s++){
        int col = 0;
        for (int p = 0; p < output_p; p++){
          for (int q = 0; q < output_q; q++){
            toep_input[INDEX_2D_TO_1D(row, col, new_input_h, new_input_w)] = input[INDEX_4D_TO_1D(0,inc, p + r, q + s, 0, weight_i, input_h, input_w)];
            col++;
          }
        }
        row++;
      }
    }
  }

  // toeplitz weight matrix in row-major
  row = 0;
  for (int m = 0; m < new_weight_r; m++){
    int col = 0;
    for (int inc = 0; inc < weight_i; inc++){
      for (int r = 0; r < weight_r; r++) {
        for (int s = 0; s < weight_s; s++){
          toep_weight[INDEX_2D_TO_1D(row, col, new_weight_r, new_weight_s)] = weight[INDEX_4D_TO_1D(m,inc, r, s, new_weight_r, weight_i, weight_r, weight_s)];
          col++;
        }
      }
    }
    row++;
  }

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toeplitz matrices, you can create a temp
  // buffer to store output_toeplitz measure runtime of this code only
  // Allocate memory for temp_output using the provided function
  float* temp_output = allocateAlignedFloatArray(opSize);
  matMul(toep_weight, toep_input, temp_output, new_weight_r, new_weight_s, new_input_h, new_input_w);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toeplitz y into row major

  //printSumOfOutputs(output, output_c * output_p * output_q);
  printSumOfOutputs(temp_output, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);
  deallocateAlignedFloatArray(temp_output);
  return 0;
}


## Q3
For `q3_conv2d_toeplitz_tranB.cc`, repeat procedures in Q2, but store kernel toeplitz matrix in column major and modify `matMul` function accordingly. Measure and report the runtime and memory usage.

In [ ]:
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

void matMulTransB(const float* A, const float* B, float* C, int A_rows, int A_cols,
            int B_rows, int B_cols) {
  if (A_cols != B_rows) {
    // The matrices can't be multiplied if A's number of columns
    // isn't equal to B's number of rows.
    throw std::invalid_argument(
        "Matrix dimensions mismatch for multiplication");
  }

  int cnt = 0;
  // TODO: implement matmul using 3 loops
  for (int i = 0; i < A_rows; i++){
    for (int k = 0; k < B_cols; k++){
      for (int j = 0; j < A_cols; j++){
        C[cnt] += A[INDEX_2D_TO_1D(j,i,0,A_rows)] * B[INDEX_2D_TO_1D(k,j,0,B_rows)];
      }
      cnt++;
    }
  }
}

int main(void) {
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = 55 * 1024 * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float* cacheFlush = allocateAlignedFloatArray(cacheSize);

  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float* weight = allocateAlignedFloatArray(weightSize);
  float* input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_c = weight_o;
  const unsigned int output_p = input_h - weight_r + 1;
  const unsigned int output_q = input_w - weight_s + 1;
  const unsigned int output_row = weight_o;
  const unsigned int output_col = output_p * output_q;
  const unsigned int opSize = output_row * output_col;
  float* output = allocateAlignedFloatArray(opSize);

  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int new_input_h = weight_r * weight_s * weight_i;
  const unsigned int new_input_w = output_p * output_q;
  const unsigned int new_ipSize = new_input_h * new_input_w;
  float* new_input = allocateAlignedFloatArray(new_ipSize);

  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int new_weight_r = weight_o;
  const unsigned int new_weight_s = new_input_h;
  const unsigned int new_weightSize = new_weight_r * new_weight_s;
  float* new_weight = allocateAlignedFloatArray(new_weightSize);

  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;

  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in column
  // major order
  // input matrix in column-major
  int cnt = 0;
  int col = 0;
  for (int p = 0; p < output_p; p++){
    for (int q = 0; q < output_q; q++){
      for (int inc = 0; inc < weight_i; inc++){
        for (int r = 0; r < weight_r; r++) {
          for (int s = 0; s < weight_s; s++){
            new_input[cnt] = input[INDEX_4D_TO_1D(0,inc, p + r, q + s, 0, weight_i, input_h, input_w)];
            cnt++;
          }
        }
      }
    }
  }

  // weight matrix in column-major
  col = 0;
  for (int inc = 0; inc < weight_i; inc++){
    for (int r = 0; r < weight_r; r++) {
      for (int s = 0; s < weight_s; s++){
        for (int m = 0; m < new_weight_r; m++){
          new_weight[col] = weight[INDEX_4D_TO_1D(m,inc, r, s, new_weight_r, weight_i, weight_r, weight_s)];
          col++;
        }
      }
    }
  }

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toeplitz matrices, you can create a temp
  // buffer to store output_toeplitz measure runtime of this code only
  // Allocate memory for temp_output using the provided function
  matMulTransB(new_weight, new_input, output, new_weight_r, new_weight_s, new_input_h, new_input_w);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toeplitz y into row major

  printSumOfOutputs(output, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);
  //deallocateAlignedFloatArray(temp_output);
  return 0;
}


## Q4
For ``q4_conv2d_toeplitz_avx.cc``, use Intel AVX (Advanced Vector Extensions) instruction set to perform matmul operation. Intel AVX instructions are Single Instruction Multiple Data (SIMD) instructions that can process 8 floating-point operands in a single instruction. Store input toeplitz matrix in row major and kernel toeplitz matrix in column major. An example of using Intel AVX is given in file ``examples/example_vectorsum_simd.cc``. Measure and report runtime and memory usage.

Hint 1: Use `_mm256_add_ps` and `_mm256_mul_ps` instructions.

Hint 2: If a vector is not divisible by 8, the remaining elements in the vector should not be processed by SIMD instruction. Instead, use normal scalar operations.

In [ ]:
#include <immintrin.h>
#include <chrono>
#include <iostream>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

// Assume row-major for A and column-major for B
void matMulAVXTransB(const float* A, const float* B, float* C, int A_rows,
                     int A_cols, int B_rows, int B_cols) {
  if (A_cols != B_rows) {
    throw std::invalid_argument(
        "Matrix dimensions mismatch for multiplication");
  }

  // TODO: implement AVX version of matMulTransB ///?????????

  // Check if A_cols is divisible by 8 for efficient AVX processing
  //__m256 sum = _mm256_setzero_ps();
    for (int i = 0; i < A_rows; i++) {
      for (int j = 0; j < B_cols; j++) {
        __m256 sum = _mm256_setzero_ps();
        float out[8];
        int k;
        float temp_out = 0;
        for (k = 0; k < A_cols-8; k += 8) {
          __m256 a = _mm256_loadu_ps(&A[i * A_cols + k]);
          __m256 b = _mm256_loadu_ps(&B[j * B_rows + k]);
          sum = _mm256_add_ps(sum, _mm256_mul_ps(a, b));
        }
        _mm256_storeu_ps(out, sum);
        temp_out = out[0] + out[1]+out[2]+out[3]+out[4]+out[5]+out[6]+out[7];
        //cout << k;
        // check :
        float val = temp_out;
        //cout << " here";
        for (; k < A_cols; k++) {
              val +=
                    A[INDEX_2D_TO_1D(i, k, A_rows, A_cols)] * B[INDEX_2D_TO_1D(j,k, B_cols, A_cols)];
        }

    C[INDEX_2D_TO_1D(i, j, A_rows, B_cols)]=val;
  }
}
}


int main(void) {
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = 72 * 1024 * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float* cacheFlush = allocateAlignedFloatArray(cacheSize);

  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;



  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float* weights = allocateAlignedFloatArray(weightSize);
  float* inputs = allocateAlignedFloatArray(inputSize);
  //float* output = allocateAlignedFloatArray(output_c*output_p*output_q);

  // init input and weight buffers
  initializeBuffer(inputs, inputSize);
  initializeBuffer(weights, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toep output and allocate buffer
  const unsigned int output_p = (input_h - weight_r + 1);
  const unsigned int output_q = (input_w - weight_s + 1);
  const unsigned int output_c = weight_o;
  const unsigned int outputToepRows = (input_h - weight_r + 1) * (input_w - weight_s + 1);
  const unsigned int outputToepCols =  weight_o;//weight_i * weight_r * weight_s;????
  const unsigned int outputToepSize = outputToepRows*outputToepCols;
  float* temp_outputs = allocateAlignedFloatArray(outputToepRows * outputToepCols);

  // TODO: calculate dimension of toep input and allocate buffer
  const unsigned int inputToepRows = (input_h - weight_r + 1) * (input_w - weight_s + 1);
  const unsigned int inputToepCols = weight_i * weight_r * weight_s;
  float* toepInputs = allocateAlignedFloatArray(inputToepRows * inputToepCols);

  // TODO: calculate dimension of toep weight and allocate buffer
  const unsigned int weightToepRows = weight_i * weight_r * weight_s;
  const unsigned int weightToepCols = weight_o;
  float* toepWeights = allocateAlignedFloatArray(weightToepRows * weightToepCols);

  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;

  // TODO: convert matrix input & weight into toep matrices
  // toep input is stored in row major and toep weight is stored in column
  // major order
  // Convert input matrix to Toeplitz format (row major)
// Compute Toeplitz input matrix
// Compute Toeplitz input matrix
for (int i = 0; i < input_h - weight_r + 1; i++) {
    for (int j = 0; j < input_w - weight_s + 1; j++) {
        for (int c = 0; c < input_c; c++) {
            for (int r = 0; r < weight_r; r++) {
                for (int s = 0; s < weight_s; s++) {
                    //unsigned int row_idx = i * (input_w - weight_s + 1) + j;
                    //unsigned int col_idx = (c * weight_r * weight_s) + (r * weight_s) + s;
                    toepInputs[INDEX_2D_TO_1D(i * (input_w - weight_s + 1) + j,(c * weight_r * weight_s) + (r * weight_s) + s,inputToepRows,inputToepCols)]=
                       inputs[INDEX_4D_TO_1D(0, c, (i + r),(j + s),1, input_c, input_h,input_w)];

                }
            }
        }
    }
}
// Compute Toeplitz weight matrix


    for (int c = 0; c < weight_i; c++) {
        for (int r = 0; r < weight_r; r++) {
            for (int s = 0; s < weight_s; s++) {
                for (int o = 0; o < weight_o; o++)

                toepWeights[INDEX_2D_TO_1D(o,c * weight_r * weight_s + r * weight_s + s,weightToepCols, weightToepRows)] =
                        weights[INDEX_4D_TO_1D(o,c,r, s, weight_o, weight_i, weight_r, weight_s)];
            }
        }
    }
  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);
  //float* temp_outputs = allocateAlignedFloatArray(outputToepSize);
  auto start = high_resolution_clock::now();


  // TODO: compute matmul between toep matrices, you can create a temp
  // buffer to store output_toep measure runtime of this code only
  // Allocate memory for temp_output using the provided function

  matMulAVXTransB(toepInputs,toepWeights, temp_outputs,  inputToepRows, inputToepCols, weightToepRows, weightToepCols);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  std::cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toep y into col major
  float* outputs = allocateAlignedFloatArray(outputToepRows * outputToepCols);
  for (unsigned int i = 0; i < outputToepRows; ++i) {
    for (unsigned int j = 0; j < outputToepCols; ++j) {
        outputs[j * outputToepRows + i] = temp_outputs[i * outputToepCols + j];
    }
}

  printSumOfOutputs(outputs, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(cacheFlush);
  deallocateAlignedFloatArray(inputs);
  deallocateAlignedFloatArray(weights);
  deallocateAlignedFloatArray(outputs);
  deallocateAlignedFloatArray(temp_outputs);
  deallocateAlignedFloatArray(toepWeights);
  deallocateAlignedFloatArray(toepInputs);
  return 0;
}



## Q5

### Part 1
In file ``q5_conv2d_toeplitz_avx_openmp.cc``, futher optimize the AVX matmul operation using ``OpenMP`` library, which enables multi-threaded parallel computing automatically through a simple and flexible interface. An example of using ``OpenMP`` is given in file ``examples/example_vectorsum_simd_omd.cc``. Run the code with different number of threads. Measure and report the runtime for number of threads = [2, 4, 8].


### Part 2 (Optional - No Credit)

In file ``q5_optional_conv2d_toeplitz_avx_multi_thread.cc``, instead of using `OpenMP`, use the C++ standard `<thread>` library to implement multi-threaded AVX matmul operation. You can refer to [this video](https://youtu.be/3aqxaZsvn80?si=1QEE580e2vLmrqPO) to learn about multi threading in C++.


In [ ]:
#include <immintrin.h>
#include <omp.h>
#include <chrono>
#include <iostream>
#include <vector>
#include "../utils.h"

using namespace std;
using namespace std::chrono;

// Assume row-major for A and column-major for B
void matMulAVXOpenMPTransB(const float* A, const float* B, float* C, int A_rows,
                           int A_cols, int B_rows, int B_cols) {
  if (A_cols != B_rows) {
    throw std::invalid_argument(
        "Matrix dimensions mismatch for multiplication");
  }

  // TODO: implement AVX version of matMulTransB with OpenMP
  // Parallelize the matrix multiplication using OpenMP
  #pragma omp parallel for shared(A, B, C) collapse(2)
  for (int i = 0; i < A_rows; i++) {
    for (int j = 0; j < B_cols; j++) {
      __m256 sum = _mm256_setzero_ps();
      float out[8];
      int k;
      float temp_out = 0;
      for (k = 0; k < A_cols - 8; k += 8) {
        __m256 a = _mm256_loadu_ps(&A[i * A_cols + k]);
        __m256 b = _mm256_loadu_ps(&B[j * B_rows + k]);
        sum = _mm256_add_ps(sum, _mm256_mul_ps(a, b));
      }
      _mm256_storeu_ps(out, sum);
      temp_out = out[0] + out[1] + out[2] + out[3] + out[4] + out[5] + out[6] + out[7];

      float val = temp_out;

      for (; k < A_cols; k++) {
        val += A[INDEX_2D_TO_1D(i, k, A_rows, A_cols)] * B[INDEX_2D_TO_1D(j, k, B_cols, A_cols)];
      }

      C[INDEX_2D_TO_1D(i, j, A_rows, B_cols)] = val;
    }
  }
}

int main(void) {
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = 55 * 1024 * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float* cacheFlush = allocateAlignedFloatArray(cacheSize);

  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float* weight = allocateAlignedFloatArray(weightSize);
  float* input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toeplitz output and allocate buffer
  const unsigned int output_p = (input_h - weight_r + 1);
  const unsigned int output_q = (input_w - weight_s + 1);
  const unsigned int output_c = weight_o;
  const unsigned int outputToepRows = (input_h - weight_r + 1) * (input_w - weight_s + 1);
  const unsigned int outputToepCols = weight_o;
  const unsigned int outputToepSize = outputToepRows * outputToepCols;
  float* temp_outputs = allocateAlignedFloatArray(outputToepRows * outputToepCols);

  // TODO: calculate dimension of toeplitz input and allocate buffer
  const unsigned int inputToepRows = (input_h - weight_r + 1) * (input_w - weight_s + 1);
  const unsigned int inputToepCols = weight_i * weight_r * weight_s;
  float* toepInputs = allocateAlignedFloatArray(inputToepRows * inputToepCols);

  // TODO: calculate dimension of toeplitz weight and allocate buffer
  const unsigned int weightToepRows = weight_i * weight_r * weight_s;
  const unsigned int weightToepCols = weight_o;
  float* toepWeights = allocateAlignedFloatArray(weightToepRows * weightToepCols);

  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;

  // TODO: convert matrix input & weight into toeplitz matrices
  // toeplitz input is stored in row major and toeplitz weight is stored in
  // column major order
  // Compute Toeplitz input matrix
  for (int i = 0; i < input_h - weight_r + 1; i++) {
    for (int j = 0; j < input_w - weight_s + 1; j++) {
      for (int c = 0; c < input_c; c++) {
        for (int r = 0; r < weight_r; r++) {
          for (int s = 0; s < weight_s; s++) {
            toepInputs[INDEX_2D_TO_1D(i * (input_w - weight_s + 1) + j,
                                      (c * weight_r * weight_s) + (r * weight_s) + s,
                                      inputToepRows, inputToepCols)] =
                input[INDEX_4D_TO_1D(0, c, (i + r), (j + s), 1, input_c, input_h, input_w)];
          }
        }
      }
    }
  }

  // Compute Toeplitz weight matrix
  for (int c = 0; c < weight_i; c++) {
    for (int r = 0; r < weight_r; r++) {
      for (int s = 0; s < weight_s; s++) {
        for (int o = 0; o < weight_o; o++) {
          toepWeights[INDEX_2D_TO_1D(o, c * weight_r * weight_s + r * weight_s + s,
                                      weightToepCols, weightToepRows)] =
              weight[INDEX_4D_TO_1D(o, c, r, s, weight_o, weight_i, weight_r, weight_s)];
        }
      }
    }
  }

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  int num_threads[] = {2, 4, 8}; // Test with 2, 4, and 8 threads

  for (int i = 0; i < 3; ++i) {
    omp_set_num_threads(num_threads[i]); // Set the number of threads
    auto start = high_resolution_clock::now();
    matMulAVXOpenMPTransB(toepInputs, toepWeights, temp_outputs, inputToepRows, inputToepCols, weightToepRows, weightToepCols);

    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<milliseconds>(stop - start);
    cout << "Time taken by function with " << num_threads[i] << " threads: " << duration.count() << " milliseconds" << endl;

    // TODO: reformat toeplitz y into row major
    float* outputs = allocateAlignedFloatArray(outputToepRows * outputToepCols);
    for (unsigned int i = 0; i < outputToepRows; ++i) {
      for (unsigned int j = 0; j < outputToepCols; ++j) {
        outputs[j * outputToepRows + i] = temp_outputs[i * outputToepCols + j];
      }
    }

    printSumOfOutputs(outputs, output_c * output_p * output_q);
    deallocateAlignedFloatArray(outputs);
  }

  // free all allocated buffers
  deallocateAlignedFloatArray(cacheFlush);
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(temp_outputs);
  deallocateAlignedFloatArray(toepWeights);
  deallocateAlignedFloatArray(toepInputs);

  return 0;
}


## Q6

In file ``q6_conv2d_toeplitz_blas.cc``, use `BLAS` library to implement matmul operation. You can decide the storage format for input and kernel toeplitz matrices, and make sure you set input arguments of `cblas_sgemm` accordingly. An example is given in file ``examples/example_matmul_blas.cc``. Search online document of `cblas_sgemm` API if you are unclear about the mearning of each of its input argument. Follow the example and and finish your own code. Measure and report runtime and memory usage.

In [ ]:
#include <immintrin.h>

#include <chrono>
#include <iostream>

#include "../utils.h"
extern "C" {
#include "../cblas.h"
}
using namespace std;
using namespace std::chrono;

void matMulBLAS(const float* A, const float* B, float* C, int A_rows,
                int A_cols, int B_rows, int B_cols) {
  if (A_cols != B_rows) {
    throw std::invalid_argument(
        "Matrix dimensions mismatch for multiplication");
  }

  //TODO: call cblas_sgemm to perform matrix multiplication
  // Call cblas_sgemm to perform matrix multiplication
  cblas_sgemm(CblasRowMajor, CblasNoTrans, CblasNoTrans, A_rows, B_cols,
              A_cols, 1.0f, A, A_cols, B, B_cols, 0.0f, C, B_cols);
}

int main(void) {
  // modify the cachesize to match your system configuration in Google Colab
  const unsigned int cacheSize = 55 * 1024 * 1024;
  const unsigned int cacheFloatSize = cacheSize / sizeof(float);

  float* cacheFlush = allocateAlignedFloatArray(cacheSize);

  // dimension of the convolution weight
  const unsigned int weight_o = 128;
  const unsigned int weight_i = 128;
  const unsigned int weight_r = 3;
  const unsigned int weight_s = 3;

  // dimension of the convolution input
  const unsigned int input_c = 128;
  const unsigned int input_h = 128;
  const unsigned int input_w = 128;

  // Calculate the sizes needed for the arrays
  const unsigned int weightSize = weight_o * weight_i * weight_r * weight_s;
  const unsigned int inputSize = input_c * input_h * input_w;

  // Allocate memory for the arrays using the provided functions
  float* weight = allocateAlignedFloatArray(weightSize);
  float* input = allocateAlignedFloatArray(inputSize);

  // init input and weight buffers
  initializeBuffer(input, inputSize);
  initializeBuffer(weight, weightSize);

  // Declare variables to store the initial memory usage values.
  double initialVirtualMemoryUsage, initialResidentSetSize;

  // Declare variables to store any subsequent memory usage values (to be used
  // later if needed).
  double subsequentVirtualMemoryUsage, subsequentResidentSetSize;

  // Call the function to get the current memory usage and store the values in
  // the initial variables.
  getMemoryUsage(initialVirtualMemoryUsage, initialResidentSetSize);

  // TODO: calculate dimension of toep output and allocate buffer
  const unsigned int outputToepRows = weight_o;
  const unsigned int outputToepCols = (input_h - weight_r + 1) * (input_w - weight_s + 1);
  const unsigned int outputToepSize = outputToepRows * outputToepCols;
  float* temp_output = allocateAlignedFloatArray(outputToepRows * outputToepCols);

  // TODO: calculate dimension of toep input and allocate buffer
  const unsigned int inputToepRows = weight_i * weight_r * weight_s;
  const unsigned int inputToepCols = (input_h - weight_r + 1) * (input_w - weight_s + 1);
  float* toepInput = allocateAlignedFloatArray(inputToepRows * inputToepCols);

  // TODO: calculate dimension of toep weight and allocate buffer
  const unsigned int weightToepRows = weight_o;
  const unsigned int weightToepCols = weight_i * weight_r * weight_s;
  float* toepWeight = allocateAlignedFloatArray(weightToepRows * weightToepCols);

  // measure memory usage for 3 buffers above only
  getMemoryUsage(subsequentVirtualMemoryUsage, subsequentResidentSetSize);
  std::cout << "Virtual Memory (KB): "
            << subsequentVirtualMemoryUsage - initialVirtualMemoryUsage
            << std::endl;

  // TODO: convert matrix input & weight into toep matrices
  // Compute Toep input matrix
  int toepRow = 0;
  for (unsigned int c = 0; c < input_c; ++c) {
    for (unsigned int r = 0; r < weight_r; ++r) {
      for (unsigned int s = 0; s < weight_s; ++s) {
        int toepCol = 0;
        for (unsigned int i = 0; i < input_h - weight_r + 1; ++i) {
          for (unsigned int j = 0; j < input_w - weight_s + 1; ++j) {
            // Compute Toep indices
            toepInput[INDEX_2D_TO_1D(toepRow, toepCol, inputToepRows, inputToepCols)] =
                input[INDEX_4D_TO_1D(0, c, i + r, j + s, 0, weight_i, input_h, input_w)];
            toepCol++;
          }
        }
        toepRow++;
      }
    }
  }

  // Compute Toep weight matrix
  toepRow = 0;
  for (unsigned int o = 0; o < weight_o; ++o) {
    int toepCol = 0;
    for (unsigned int c = 0; c < weight_i; ++c) {
      for (unsigned int r = 0; r < weight_r; ++r) {
        for (unsigned int s = 0; s < weight_s; ++s) {
          toepWeight[INDEX_2D_TO_1D(toepRow, toepCol, weightToepRows, weightToepCols)] =
              weight[INDEX_4D_TO_1D(o, c, r, s, weight_o, weight_i, weight_r, weight_s)];
          toepCol++;
        }
      }
    }
    toepRow++;
  }

  // Flush cache
  flushCache(cacheFlush, cacheFloatSize);

  auto start = high_resolution_clock::now();
  // TODO: compute matmul between toep matrices, you can create a temp
  // buffer to store output_toep measure runtime of this code only
  // Allocate memory for temp_output using the provided function
  matMulBLAS(toepWeight, toepInput, temp_output, weightToepRows, weightToepCols, inputToepRows, inputToepCols);

  auto stop = high_resolution_clock::now();
  auto duration = duration_cast<milliseconds>(stop - start);
  cout << "Time taken by function: " << duration.count() << " milliseconds"
       << endl;

  // TODO: reformat toep y into row major
  float* output = allocateAlignedFloatArray(outputToepRows * outputToepCols);
  for (unsigned int i = 0; i < outputToepRows; ++i) {
    for (unsigned int j = 0; j < outputToepCols; ++j) {
      output[i * outputToepCols + j] = temp_output[j * outputToepRows + i];
    }
  }
  const unsigned int output_p = (input_h - weight_r + 1);
  const unsigned int output_q = (input_w - weight_s + 1);
  const unsigned int output_c = weight_o;
  printSumOfOutputs(temp_output, output_c * output_p * output_q);

  // free all allocated buffers
  deallocateAlignedFloatArray(cacheFlush);
  deallocateAlignedFloatArray(input);
  deallocateAlignedFloatArray(weight);
  deallocateAlignedFloatArray(output);
  deallocateAlignedFloatArray(temp_output);
  deallocateAlignedFloatArray(toepInput);
  deallocateAlignedFloatArray(toepWeight);
  return 0;
}


## Q7  

### Part 1


Analyze the performance differences between Q1-Q6. Explain what constitutes the performance difference between each implementations and why `BLAS` library is super fast.

### Part 2: Using Google Benchmark (Optional - No Credit)
As an optional practice, you can measure the runtime of each matrix calculation method more accurately using Google benchmark.

A better way to measure performance of a funcion in C++ is to use Google Benchmark. You can refer to [this video](https://youtu.be/9VKR8u9odrA?si=xSInuzT5uMBOKAbP) to familiarize yourself with this package.




## Upload files to GitHub
Make sure upload your final C++ code and this IPython notebook to GitHub Repo either mannully or through git commands.